In [ ]:
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import classification_report, accuracy_score

In [ ]:
dataset_path = 'Data/Processed/Image/'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

# Load training data with augmentation
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(608, 456),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(608, 456),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

In [ ]:
# Define your hypermodel using Keras Tuner (example with RandomSearch)
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int('conv_1_units', min_value=32, max_value=128, step=32),
                     (3, 3), activation='relu', input_shape=(608, 456, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32),
                    activation='relu'))
    model.add(Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid')) # For binary classification
    model.compile(optimizer=SGD(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',  # Or other relevant metric
    max_trials=5,  # Number of hyperparameter combinations to try
    directory='my_dir',  # Directory to store results
    project_name='my_project'
)

# Search for best hyperparameters
tuner.search(train_generator,
             epochs=10,
             validation_data=validation_generator)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the best model and retrain it
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
               ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)]
)

In [ ]:
plt.figure(figsize=(14, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.show()